In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import datetime

In [23]:
header ={
        'scheme':'https',
        'Accept':'application/json, text/plain, */*',
        'Accept-Language':'en-US,en;q=0.8',
        'Referer':'https://www.bseindia.com/',
        'Sec-Ch-Ua':'"Not_A Brand";v="8", "Chromium";v="120", "Brave";v="120"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform':'"Windows"',
        'Sec-Fetch-Mode':'cors',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }

In [24]:
chrome_options = Options()
chrome_options.add_argument('--headless')
for key, value in header.items():
    chrome_options.add_argument(f"{key}={value}")
driver = webdriver.Chrome(options=chrome_options)
#driver.maximize_window()
url = 'https://www.bseindia.com/markets/MarketInfo/NoticesCirculars.aspx?id=0&txtscripcd=&pagecont=&subject='
driver.get(url)

In [25]:
Date = (datetime.datetime.now() - datetime.timedelta(days = 7 )).strftime('%d')
Month = (datetime.datetime.now() - datetime.timedelta(days = 7 )).strftime('%b')
if Date.startswith('0'):
    Date = Date[1:]

print(Date)
print(Month)

8
Feb


In [26]:
driver.find_element(By.ID, 'ContentPlaceHolder1_txtDate').click()
driver.find_element(By.CLASS_NAME,'ui-datepicker-month').click()
driver.find_element(By.XPATH,f"//option[text()='{Month}']").click()
driver.find_element(By.XPATH,f"//a[text()='{Date}']").click()
dropdown = driver.find_element(By.ID, "ContentPlaceHolder1_ddlDep")
select = Select(dropdown)  
select.select_by_visible_text('Listing Operations')
element = driver.find_element(By.ID,'ContentPlaceHolder1_btnSubmit')
ActionChains(driver).click(element).perform()
wait = WebDriverWait(driver, 10)
time.sleep(1)
PageNo = []
DataList = []
Links_List = [] 
r = driver.page_source
print("1")
soup = bs(r,'html.parser')
Table = soup.find('table',{'id':'ContentPlaceHolder1_GridView2'})
Rows = Table.find_all('tr')
for row in Rows[-1:]:
    Datas = row.find_all('td')
    for data in Datas:
        PageNo.append(data.text.strip())
for row in Rows[1:21]:
        Datas = row.find_all('td')
        link = f'https://www.bseindia.com{row.find('a').get('href')}'
        Links_List.append(link)
        for data in Datas[:5]:
            DataList.append(data.text.strip('\n'))
            
for i in range(2,len(PageNo)):
    element = wait.until(EC.element_to_be_clickable((By.XPATH, f"//a[text()='{i}']")))
    driver.execute_script("arguments[0].scrollIntoView();", element)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(5)
    r = driver.page_source
    print("2")
    soup = bs(r,'html.parser')
    Table = soup.find('table',{'id':'ContentPlaceHolder1_GridView2'})
    Rows = Table.find_all('tr')
    for row in Rows[1:21]:
        Datas = row.find_all('td')
        link = f'https://www.bseindia.com{row.find('a').get('href')}'
        Links_List.append(link)
        for data in Datas[:5]:
            DataList.append(data.text.strip('\n'))
            
            
        

1
2
2
2
2
2
2
2


In [27]:
BSE_Circular = []
for i in range(0,len(DataList),5):
    PDFormate = DataList[i:i+5]
    BSE_Circular.append(PDFormate)
df = pd.DataFrame(BSE_Circular,columns=['Date','Subject','Segment Name','Category Name','Department'])
df['Links'] = Links_List

In [28]:
Filtered_df = df[df['Subject'].str.contains('Listing of Equity')]
Filtered_df

,Date,Subject,Segment Name,Category Name,Department,Links
13,20240214-39,Listing of Equity Shares of Rudra Gas Enterpri...,SME,Company related,Listing Operations,https://www.bseindia.com/markets/MarketInfo/Di...
28,20240214-8,Listing of Equity Shares of Rudra Gas Enterpri...,SME,Company related,Listing Operations,https://www.bseindia.com/markets/MarketInfo/Di...
42,20240213-47,Listing of Equity Shares of Jana Small Finance...,Equity,Company related,Listing Operations,https://www.bseindia.com/markets/MarketInfo/Di...
43,20240213-46,Listing of Equity Shares of Capital Small Fina...,Equity,Company related,Listing Operations,https://www.bseindia.com/markets/MarketInfo/Di...
44,20240213-45,Listing of Equity Shares of Rashi Peripherals ...,Equity,Company related,Listing Operations,https://www.bseindia.com/markets/MarketInfo/Di...
66,20240213-12,Listing of Equity Shares of Capital Small Fina...,Equity,Company related,Listing Operations,https://www.bseindia.com/markets/MarketInfo/Di...
67,20240213-10,Listing of Equity Shares of Rashi Peripherals ...,Equity,Company related,Listing Operations,https://www.bseindia.com/markets/MarketInfo/Di...
68,20240213-9,Listing of Equity Shares of Jana Small Finance...,Equity,Company related,Listing Operations,https://www.bseindia.com/markets/MarketInfo/Di...
83,20240212-31,Listing of Equity Shares of Signpost India Lim...,Equity,Company related,Listing Operations,https://www.bseindia.com/markets/MarketInfo/Di...
122,20240209-43,Listing of Equity Shares of Apeejay Surrendra ...,Equity,Company related,Listing Operations,https://www.bseindia.com/markets/MarketInfo/Di...


In [29]:
driver.close()